In [8]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 58.8 MB/s eta 0:00:00


In [1]:
!pip install cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.7/249.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00


In [23]:
!pip install rank_bm25

In [2]:
import pandas as pd
import numpy as np
import cohere
from tqdm import tqdm

In [3]:
cohere_api = "BnsT8NOSeniMEC7DKicLEewR17aaJTa4uhR7X2eG"

co = cohere.Client(cohere_api)

In [4]:
text = """
Interstellar is a 2014 epic science fiction film co-written,
directed, and produced by Christopher Nolan.
It stars Matthew McConaughey, Anne Hathaway, Jessica Chastain,
Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine.
Set in a dystopian future where humanity is struggling to
survive, the film follows a group of astronauts who travel
through a wormhole near Saturn in search of a new home for
mankind.
Brothers Christopher and Jonathan Nolan wrote the screenplay,
which had its origins in a script Jonathan developed in 2007.
Caltech theoretical physicist and 2017 Nobel laureate in
Physics[4] Kip Thorne was an executive producer, acted as a
scientific consultant, and wrote a tie-in book, The Science of
Interstellar.
Cinematographer Hoyte van Hoytema shot it on 35 mm movie film in
the Panavision anamorphic format and IMAX 70 mm.
Principal photography began in late 2013 and took place in
Alberta, Iceland, and Los Angeles.
Interstellar uses extensive practical and miniature effects and
the company Double Negative created additional digital effects.
Interstellar premiered on October 26, 2014, in Los Angeles.
In the United States, it was first released on film stock,
expanding to venues using digital projectors.
The film had a worldwide gross over $677 million (and $773
million with subsequent re-releases), making it the tenth-highest
grossing film of 2014.
It received acclaim for its performances, direction, screenplay,
musical score, visual effects, ambition, themes, and emotional
weight.
It has also received praise from many astronomers for its
scientific accuracy and portrayal of theoretical astrophysics.
Since its premiere, Interstellar gained a cult following,[5] and
now is regarded by many sci-fi experts as one of the best
science-fiction films of all time.
Interstellar was nominated for five awards at the 87th Academy
Awards, winning Best Visual Effects, and received numerous other
accolades
"""

In [5]:
texts = text.split(".")
texts = [t.strip(' \n') for t in texts]

texts[:5]

['Interstellar is a 2014 epic science fiction film co-written,\ndirected, and produced by Christopher Nolan',
 'It stars Matthew McConaughey, Anne Hathaway, Jessica Chastain,\nBill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine',
 'Set in a dystopian future where humanity is struggling to\nsurvive, the film follows a group of astronauts who travel\nthrough a wormhole near Saturn in search of a new home for\nmankind',
 'Brothers Christopher and Jonathan Nolan wrote the screenplay,\nwhich had its origins in a script Jonathan developed in 2007',
 'Caltech theoretical physicist and 2017 Nobel laureate in\nPhysics[4] Kip Thorne was an executive producer, acted as a\nscientific consultant, and wrote a tie-in book, The Science of\nInterstellar']

In [6]:
# get the embeddings

response = co.embed(
    texts=texts,
    input_type="search_document"
)

embeddings = np.array(response.embeddings)
embeddings.shape

(15, 4096)

In [10]:
import faiss

dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
print(index.is_trained)
index.add(np.float32(embeddings))

True


In [11]:
def search(query, num_results=3):
    query_embedding = co.embed(
        texts=[query],
        input_type="search_query"
    ).embeddings[0]

    distances, similar_items = index.search(np.float32([query_embedding]), num_results)
    texts_np = np.array(texts)
    results = pd.DataFrame(
        data={
            "text": texts_np[similar_items[0]],
            "distance": distances[0]
        }
    )

    print(f"Query : {query} \nNearest neighbors: ")
    return results

In [12]:
query = "how precise was the science?"

results = search(query)
results

Query : how precise was the science? 
Nearest neighbors: 


,text,distance
0,It has also received praise from many astronom...,9659.718750
1,Interstellar uses extensive practical and mini...,10615.988281
2,Caltech theoretical physicist and 2017 Nobel l...,10969.982422


In [13]:
results.iloc[0]["text"]

'It has also received praise from many astronomers for its\nscientific accuracy and portrayal of theoretical astrophysics'

In [14]:
results.iloc[1]["text"]

'Interstellar uses extensive practical and miniature effects and\nthe company Double Negative created additional digital effects'

In [15]:
results.iloc[2]["text"]

'Caltech theoretical physicist and 2017 Nobel laureate in\nPhysics[4] Kip Thorne was an executive producer, acted as a\nscientific consultant, and wrote a tie-in book, The Science of\nInterstellar'

In [16]:
query = "What is the mass of the moon?"

results = search(query)
results

Query : What is the mass of the moon? 
Nearest neighbors: 


,text,distance
0,The film had a worldwide gross over $677 milli...,12916.307617
1,Cinematographer Hoyte van Hoytema shot it on 3...,13106.837891
2,"It received acclaim for its performances, dire...",13191.150391


In [17]:
# rerank



query = "how precise was the science"
results = co.rerank(
    query=query,
    documents=texts,
    top_n=3,
    return_documents=True
)

results.results

[RerankResponseResultsItem(document=RerankResponseResultsItemDocument(text='It has also received praise from many astronomers for its\nscientific accuracy and portrayal of theoretical astrophysics'), index=12, relevance_score=0.16981852),
 RerankResponseResultsItem(document=RerankResponseResultsItemDocument(text='The film had a worldwide gross over $677 million (and $773\nmillion with subsequent re-releases), making it the tenth-highest\ngrossing film of 2014'), index=10, relevance_score=0.07004896),
 RerankResponseResultsItem(document=RerankResponseResultsItemDocument(text='Caltech theoretical physicist and 2017 Nobel laureate in\nPhysics[4] Kip Thorne was an executive producer, acted as a\nscientific consultant, and wrote a tie-in book, The Science of\nInterstellar'), index=4, relevance_score=0.0043994132)]

In [18]:
for idx, result in enumerate(results.results):
    print(f"Rank {idx+1}: {result.document.text}, score: {result.relevance_score}")

Rank 1: It has also received praise from many astronomers for its
scientific accuracy and portrayal of theoretical astrophysics, score: 0.16981852
Rank 2: The film had a worldwide gross over $677 million (and $773
million with subsequent re-releases), making it the tenth-highest
grossing film of 2014, score: 0.07004896
Rank 3: Caltech theoretical physicist and 2017 Nobel laureate in
Physics[4] Kip Thorne was an executive producer, acted as a
scientific consultant, and wrote a tie-in book, The Science of
Interstellar, score: 0.0043994132


In [26]:
from rank_bm25 import BM25Okapi
from sklearn.feature_extraction import _stop_words
import string

def bm25_tokenizer(text):
    tokenized_doc = []
    for token in text.lower().split():
        token = token.strip(string.punctuation)

        if len(token) > 0 and token not in _stop_words.ENGLISH_STOP_WORDS:
            tokenized_doc.append(token)

    return tokenized_doc

bm25 = BM25Okapi(
    [bm25_tokenizer(text) for text in texts]
)

In [33]:
def keyword_and_reranking_search(query, top_k=3, num_candidates=10):
    print("input question: ", query)
    bm25_scores = bm25.get_scores(bm25_tokenizer(query))
    top_n = np.argpartition(bm25_scores, -num_candidates)[-num_candidates:]
    bm25_hits = [{"corpus_id": idx, "score": bm25_scores[idx]} for idx in top_n]
    bm25_hits = sorted(bm25_hits, key=lambda x: x["score"], reverse=True)
    print(f"Top-3 lexical search (BM25) hits")

    for hit in bm25_hits[0:top_k]:
        print(hit['score'],texts[hit['corpus_id']].replace("\n", " "))

    docs = [texts[hit['corpus_id']] for hit in bm25_hits]
    print("=========\n")
    print(f" Top-3 hits by rank-API ({len(bm25_hits)} BM25 hits re-ranked)")
    results = co.rerank(
        query=query,
        documents=docs,
        top_n=top_k,
        return_documents=True
    )


    for hit in results.results:
        print(hit.relevance_score, hit.document.text.replace("\n", " "))

In [34]:
keyword_and_reranking_search(query = "how precise was the science")

input question:  how precise was the science
Top-3 lexical search (BM25) hits
1.788604950756303 Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan
1.372650311045535 Caltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar
0.0 Interstellar uses extensive practical and miniature effects and the company Double Negative created additional digital effects

 Top-3 hits by rank-API (10 BM25 hits re-ranked)
0.0043994132 Caltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar
0.0035380858 Set in a dystopian future where humanity is struggling to survive, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for mankind
0.00